# Starter Code

In [98]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [99]:
df_2018 = pd.read_csv('../data/2018.csv')
df_2017 = pd.read_csv('../data/2017.csv')
df_2016 = pd.read_csv('../data/2016.csv')

def get_classifier_convert(df):
    conditions = [
        (df['fourth_down_converted'] == 1.0),
        (df['fourth_down_failed'] == 1.0),
        (df['field_goal_attempt'] == 1.0),
        (df['punt_attempt'] == 1.0)
    ]

    results = ['CONVERTED', 'FAILED', 'FIELD_GOAL', 'PUNT']

    y = np.select(conditions, results)
    return y

def get_classifier_attempt(df):
    conditions = [
        (df['fourth_down_converted'] == 1.0) | (df['fourth_down_failed'] == 1.0),
        (df['field_goal_attempt'] == 1.0),
        (df['punt_attempt'] == 1.0)
    ]

    results = ['ATTEMPTED', 'FIELD_GOAL', 'PUNT']

    y = np.select(conditions, results)
    return y

def ready_data_convert(df):
    df = df.dropna()
    y = get_classifier_convert(df)
    df = df.drop(columns=['posteam', 'fourth_down_converted', 'fourth_down_failed', 'field_goal_attempt', 'punt_attempt', 'game_date'])
    return df, y

def ready_data_attempt(df):
    df = df.dropna()
    y = get_classifier_attempt(df)
    df = df.drop(columns=['posteam', 'fourth_down_converted', 'fourth_down_failed', 'field_goal_attempt', 'punt_attempt', 'game_date'])
    return df, y

# Tree

## Convert 4th down

In [100]:
x, y = ready_data_convert(pd.concat([df_2016, df_2017, df_2018]))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

clf = RandomForestClassifier(max_depth=7, random_state=0)
clf.fit(X_train, y_train)
print(f"Test Accuracy: {clf.score(X_test, y_test)}")
print(f"Train Accuracy: {clf.score(X_train, y_train)}")

Test Accuracy: 0.8546387345019238
Train Accuracy: 0.8695745135770794


## Attempt 4th down

In [101]:
x, y = ready_data_attempt(pd.concat([df_2016, df_2017, df_2018]))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

clf = RandomForestClassifier(max_depth=5, random_state=0)
clf.fit(X_train, y_train)
print(f"Test Accuracy: {clf.score(X_test, y_test)}")
print(f"Train Accuracy: {clf.score(X_train, y_train)}")

Test Accuracy: 0.8845660538691749
Train Accuracy: 0.8787684413085312
